Training, Validation and testing dataset were labeled by batch manually

In [1]:
import sys
sys.path.append('/host/d/Github/')
import os
import numpy as np
import pandas as pd
import nibabel as nb
import Diffusion_denoising_thin_slice.functions_collection as ff

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
patient_sheet = ff.find_all_target_files(['*'],os.path.join('/host/d/Data/low_dose_CT/nii_imgs'))
print('total patient num: ', len(patient_sheet))

total patient num:  10


In [4]:
# shuffle these patients
np.random.seed(42)
shuffled_indices = np.random.permutation(len(patient_sheet))
patient_sheet = [patient_sheet[i] for i in shuffled_indices]

In [5]:
# 
results = []
for i in range(0, len(patient_sheet)):
    patient_id = os.path.basename(patient_sheet[i])
    # first 6 are train, seven and eight are val, last three are test
    if i < 6:
        batch = 'train'
    elif i == 6:
        batch = 'val'
    else:
        batch = 'test'
    results.append([batch, patient_id])
df = pd.DataFrame(results, columns=['batch','Patient_ID'])
save_path = os.path.join('/host/d/Data/low_dose_CT/Patient_lists','mayo_low_dose_CT_batched.xlsx')
df.to_excel(save_path, index=False)

# build list for simulations

In [9]:
patient_sheet = pd.read_excel(os.path.join('/host/d/Data/low_dose_CT/Patient_lists/mayo_low_dose_CT_batched.xlsx'),dtype={'Patient_ID': str})
noise_types = ['gaussian','poisson']
simulation_num = 2

data_path = '/host/d/Data/low_dose_CT/'

for noise_type in noise_types:
    results = []
    for i in range(0, len(patient_sheet)):
        patient_id = patient_sheet['Patient_ID'][i]
        batch = patient_sheet['batch'][i]
        print(f"Processing patient {patient_id} in batch {batch} with noise type {noise_type}")


        ground_truth_file = os.path.join(data_path,'nii_imgs', patient_id, 'img.nii.gz')
        img = nb.load(ground_truth_file).get_fdata()
        slice_num = img.shape[2]
        max_value = np.max(img)
        min_value = np.min(img)
        print('ground truth max and min value:', max_value, min_value)


        for n in range(0,simulation_num):
            n_type = 'gaussian' if noise_type == 'gaussian' else 'possion'
            simulation_file_odd = os.path.join(data_path,'simulation_v1', patient_id, n_type + '_random_' + str(n), 'recon_odd.nii.gz')
            simulation_file_even = os.path.join(data_path,'simulation_v1', patient_id, n_type + '_random_' + str(n), 'recon_even.nii.gz')
            

            results.append([batch,patient_id, n, simulation_file_odd, simulation_file_even, ground_truth_file, slice_num])

        df = pd.DataFrame(results, columns=['batch','Patient_ID', 'random_num', 'simulation_file_odd', 'simulation_file_even', 'ground_truth_file', 'slice_num'])
        df.to_excel(os.path.join('/host/d/Data/low_dose_CT/Patient_lists', 'mayo_low_dose_CT_' + noise_type + '_simulation_v1.xlsx'), index=False)



Processing patient L333 in batch train with noise type gaussian
ground truth max and min value: 1667.0 -1024.0
Processing patient L096 in batch train with noise type gaussian
ground truth max and min value: 3071.0 -1024.0
Processing patient L286 in batch train with noise type gaussian
ground truth max and min value: 2660.0 -1024.0
Processing patient L067 in batch train with noise type gaussian
ground truth max and min value: 2123.0 -1024.0
Processing patient L310 in batch train with noise type gaussian
ground truth max and min value: 1420.0 -1024.0
Processing patient L109 in batch train with noise type gaussian
ground truth max and min value: 2220.0 -1024.0
Processing patient L506 in batch val with noise type gaussian
ground truth max and min value: 3071.0 -1024.0
Processing patient L192 in batch test with noise type gaussian
ground truth max and min value: 3065.0 -1024.0
Processing patient L143 in batch test with noise type gaussian
ground truth max and min value: 2951.0 -1024.0
Proce